In [1]:
# Imports
import numpy as np
import pandas as pd
import muon as mu
import mofax
import gget
import gseapy as gp

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Define the data and figure folder.
data_folder = "/users/csb/huizing/Documents/PhD/Code/mowgli_reproducibility/data/"
h_folder = "/users/csb/huizing/Documents/PhD/Code/Mowgli/local_analysis/from_jz/h/"

In [5]:
# Load the data.
mdata = mu.read_h5mu(data_folder + "TEA/tea_preprocessed.h5mu.gz")

In [6]:
# Load MOFA+'s weights.
mofa_model = mofax.mofa_model(data_folder + "TEA/tea_mofa_15.hdf5")
H_mofa = mofa_model.get_weights("rna")

In [7]:
# Load Mowgli's weights.
H_mowgli = np.load(
    h_folder + "tea_mowgli_cosine_50_0_05_rna_0_01_atac_0_1_adt_0_01_0_001.npy",
    allow_pickle=True,
).item()["H_rna"]

In [3]:
def top_mowgli(dim):
    """
    Get the top n genes for a given dimension.
    """
    idx = H_mowgli[:, dim].argsort()[::-1]
    return mdata["rna"].var_names[idx].str.replace("rna:", "").to_list()


def top_mofa(dim):
    """
    Get the top n genes for a given dimension.
    """
    idx = H_mofa[:, dim].argsort()[::-1]
    return mdata["rna"].var_names[idx].str.replace("rna:", "").to_list()

In [4]:
gmt_folder = "/users/csb/huizing/Documents/PhD/Code/mowgli_reproducibility/enrich/gmts/"
sources = [
    gmt_folder + "GO_Biological_Process_2021",
    gmt_folder + "GO_Cellular_Component_2021",
    gmt_folder + "GO_Molecular_Function_2021",
    gmt_folder + "KEGG_2021_Human",
    gmt_folder + "PanglaoDB_Augmented_2021",
    gmt_folder + "Reactome_2016",
]